# Q&A Chatbot     
Christoph Windheuser, OutSystems    
Sept. 15 2021

In [1]:
import torch                                           # Import pytorch
import textwrap                                        # Import TextWrapper to wrap text to 80 characters
from transformers import BertForQuestionAnswering      # Load the BERT QA model (300 Mio parameters, 1.3 GB)
from transformers import BertTokenizer                 # Load BERT Tokenizer

model     = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [2]:
def answer_question(question, answer_text):
  input_ids    = tokenizer.encode(question, answer_text)
  tokens       = tokenizer.convert_ids_to_tokens(input_ids)
  sep_index    = input_ids.index(tokenizer.sep_token_id)
  num_seg_a    = sep_index + 1
  num_seg_b    = len(input_ids) - num_seg_a
  segment_ids  = [0]*num_seg_a + [1]*num_seg_b
  assert len(segment_ids) == len(input_ids)

  start_scores, end_scores = model(torch.tensor([input_ids]),
                                   token_type_ids=torch.tensor([segment_ids]), return_dict=False)

  answer_start = torch.argmax(start_scores)
  answer_end   = torch.argmax(end_scores)
  answer       = tokens[answer_start]

  for i in range(answer_start + 1, answer_end + 1):
    if tokens[i][0:2] == '##':
      answer += tokens[i][2:]
    else:
      answer += ' ' + tokens[i]

  return answer 


In [3]:
wrapper = textwrap.TextWrapper(width=80)    # Wrap text to 80 characters

paragraph_text = """
A business case captures the reasoning for initiating a project or task.
It is often presented in a well-structured written document, but may also come in the form of a
short verbal agreement or presentation. The logic of the business case is that, whenever resources
such as money or effort are consumed, they should be in support of a specific business need.
An example could be that a software upgrade might improve system performance, but the business case is
that better performance would improve customer satisfaction, require less task processing time, or reduce
system maintenance costs. A compelling business case adequately captures both the quantifiable and
non-quantifiable characteristics of a proposed project.
Business cases can range from comprehensive and highly structured, as required by formal project
management methodologies, to informal and brief. Information included in a formal business case could be
the background of the project, the expected business benefits, the options considered (with reasons for
rejecting or carrying forward each option), the expected costs of the project, a gap analysis and the
expected risks. Consideration should also be given to the option of doing nothing including the costs
and risks of inactivity. From this information, the justification for the project is derived.
"""
print(wrapper.fill(paragraph_text))
print ("Length of paragraph_text: %d" % len(paragraph_text))


 A business case captures the reasoning for initiating a project or task. It is
often presented in a well-structured written document, but may also come in the
form of a short verbal agreement or presentation. The logic of the business case
is that, whenever resources such as money or effort are consumed, they should be
in support of a specific business need. An example could be that a software
upgrade might improve system performance, but the business case is that better
performance would improve customer satisfaction, require less task processing
time, or reduce system maintenance costs. A compelling business case adequately
captures both the quantifiable and non-quantifiable characteristics of a
proposed project. Business cases can range from comprehensive and highly
structured, as required by formal project management methodologies, to informal
and brief. Information included in a formal business case could be the
background of the project, the expected business benefits, the optio

In [11]:
question = "What is derived from a business case?"
answer   = answer_question(question, paragraph_text)
print('Answer: "' + answer + '"')


Answer: "justification for the project"
